In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/flood-area-segmentation'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/content/drive/MyDrive/flood-area-segmentation/metadata.csv
/content/drive/MyDrive/flood-area-segmentation/Mask/7.png
/content/drive/MyDrive/flood-area-segmentation/Mask/3095.png
/content/drive/MyDrive/flood-area-segmentation/Mask/3058.png
/content/drive/MyDrive/flood-area-segmentation/Mask/1018.png
/content/drive/MyDrive/flood-area-segmentation/Mask/3020.png
/content/drive/MyDrive/flood-area-segmentation/Mask/1078.png
/content/drive/MyDrive/flood-area-segmentation/Mask/1022.png
/content/drive/MyDrive/flood-area-segmentation/Mask/3014.png
/content/drive/MyDrive/flood-area-segmentation/Mask/1051.png
/content/drive/MyDrive/flood-area-segmentation/Mask/2032.png
/content/drive/MyDrive/flood-area-segmentation/Mask/3024.png
/content/drive/MyDrive/flood-area-segmentation/Mask/1057.png
/content/drive/MyDrive/flood-area-segmentation/Mask/1071.png
/content/drive/MyDrive/flood-area-segmentation/Mask/2048.png
/content/drive/MyDrive/flood-area-segmentation/Mask/1032.png
/content/drive/MyDrive/flood

# Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, Dropout
from tensorflow.keras.models import Model
from keras.models import *
from keras.layers import *
from keras.optimizers import *

# Importing the data

In [ ]:
base_directory = '/content/drive/MyDrive/flood-area-segmentation'
images_folder = os.path.join(base_directory, 'images')
masks_folder = os.path.join(base_directory, 'masks')
data = pd.read_csv(os.path.join(base_directory, 'metadata.csv'))

In [ ]:
data

,Image,Mask
0,0.jpg,0.png
1,1.jpg,1.png
2,2.jpg,2.png
3,3.jpg,3.png
4,4.jpg,4.png
...,...,...
285,1083.jpg,1083.png
286,1084.jpg,1084.png
287,1085.jpg,1085.png
288,1086.jpg,1086.png


# Segmentation

In [ ]:
img_dim = 256

image_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.15)
mask_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.15)

train_image_generator = image_datagen.flow_from_directory(
        #'/kaggle/input/flood-area-segmentation/Image',
        '/content/drive/MyDrive/flood-area-segmentation/Image',
        target_size=(img_dim, img_dim),
        class_mode = None,
        classes = ['images'],
        batch_size = 32,
        seed=42,
        subset='training')

train_mask_generator = mask_datagen.flow_from_directory(
        #'/kaggle/input/flood-area-segmentation/Mask',
        '/content/drive/MyDrive/flood-area-segmentation/Mask',
        target_size=(img_dim, img_dim),
        class_mode = None,
        classes = ['masks'],
        color_mode = 'grayscale',
        batch_size = 32,
        seed=42,
        subset='training')

val_image_generator = image_datagen.flow_from_directory(
        #'/kaggle/input/flood-area-segmentation/Image',
        '/content/drive/MyDrive/flood-area-segmentation/Image',
        target_size=(img_dim, img_dim),
        class_mode = None,
        classes = ['images'],
        batch_size = 32,
        seed=42,
        subset='validation')

val_mask_generator = mask_datagen.flow_from_directory(
        #'/kaggle/input/flood-area-segmentation/Mask',
        '/content/drive/MyDrive/flood-area-segmentation/Mask',
        target_size=(img_dim, img_dim),
        class_mode = None,
        classes = ['masks'],
        color_mode = 'grayscale',
        batch_size = 32,
        seed=42,
        subset='validation')

train_generator = zip(train_image_generator, train_mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)

Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


# Needed Functions

In [ ]:
def create_data(data_dir):
    image_paths = []
    mask_paths = []

    folds = sorted(os.listdir(data_dir))
    for fold in folds:
        foldpath = os.path.join(data_dir, fold)
        if fold in ['image', 'Image', 'images', 'Images', 'IMAGES']:
            images = sorted(os.listdir(foldpath))
            for image in images:
                fpath = os.path.join(foldpath, image)
                image_paths.append(fpath)

        elif fold in ['mask', 'Mask', 'masks', 'Masks', 'MASKS']:
            masks = sorted(os.listdir(foldpath))
            for mask in masks:
                fpath = os.path.join(foldpath, mask)
                mask_paths.append(fpath)
        else:
            continue

    return image_paths, mask_paths

In [ ]:
# function to read an image
def load_image(image, SIZE):
    return np.round(tf.image.resize(img_to_array(load_img(image)) / 255., (SIZE, SIZE)), 4)

In [ ]:
# function to read multiple images
def load_images(image_paths, SIZE, mask=False, trim=None):
    if trim is not None:
        image_paths = image_paths[:trim]

    if mask:
        images = np.zeros(shape=(len(image_paths), SIZE, SIZE, 1))
    else:
        images = np.zeros(shape=(len(image_paths), SIZE, SIZE, 3))

    for i, image in enumerate(image_paths):
        img = load_image(image, SIZE)
        if mask:
            images[i] = img[:, :, :1]
        else:
            images[i] = img

    return images

In [ ]:
def show_image(image, title=None, cmap=None, alpha=1):
    plt.imshow(image, cmap=cmap, alpha=alpha)
    if title is not None:
        plt.title(title)
    plt.axis('off')

In [ ]:
def show_mask(image, mask, cmap=None, alpha=0.4):
    plt.imshow(image)
    plt.imshow(tf.squeeze(mask), cmap=cmap, alpha=alpha)
    plt.axis('off')

In [ ]:
def show_images(imgs, msks):
    plt.figure(figsize=(13,8))

    for i in range(15):
        plt.subplot(3,5,i+1)
        id = np.random.randint(len(imgs))
        show_mask(imgs[id], msks[id], cmap='binary')

    plt.tight_layout()
    plt.show()

In [ ]:
#This function take training model and plot history of accuracy and losses with the best epoch in both of them.

def plot_training(hist):

    # Define needed variables
    tr_acc = hist.history['accuracy']
    tr_loss = hist.history['loss']
    val_acc = hist.history['val_accuracy']
    val_loss = hist.history['val_loss']
    index_loss = np.argmin(val_loss)
    val_lowest = val_loss[index_loss]
    index_acc = np.argmax(val_acc)
    acc_highest = val_acc[index_acc]
    Epochs = [i+1 for i in range(len(tr_acc))]
    loss_label = f'best epoch= {str(index_loss + 1)}'
    acc_label = f'best epoch= {str(index_acc + 1)}'

    # Plot training history
    plt.figure(figsize= (20, 8))
    plt.style.use('fivethirtyeight')

    plt.subplot(1, 2, 1)
    plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
    plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
    plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
    plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
    plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout
    plt.show()

# U-Net

In [ ]:
image_size = (256, 256, 3)

In [ ]:
def unet(sz=image_size):
    x = Input(sz)
    inputs = x

    #downsampling
    f = 8 #number of filters
    layers = []
    for i in range(6):
        x = Conv2D(f, 3, activation='relu',padding='same')(x)
        x = Conv2D(f, 3, activation='relu',padding='same')(x)
        layers.append(x)
        x = MaxPooling2D()(x)
        f = f*2
    ff2 = 64

    #bottleneck
    j = len(layers)-1
    x = Conv2D(f, 3, activation='relu',padding='same')(x)
    x = Conv2D(f, 3, activation='relu',padding='same')(x)
    x = Conv2DTranspose(ff2, 2, strides=(2,2),padding='same')(x)
    x = Concatenate(axis=3)([x,layers[j]])
    j = j-1

    #upsampling
    for i in range(5):
        ff2 = ff2//2
        f = f//2
        x = Conv2D(f, 3, activation='relu',padding='same')(x)
        x = Conv2D(f, 3, activation='relu',padding='same')(x)
        x = Conv2DTranspose(ff2, 2, strides=(2,2),padding='same')(x)
        x = Concatenate(axis=3)([x,layers[j]])
        j = j-1
    #classification
    x = Conv2D(f, 3, activation='relu',padding='same')(x)
    x = Conv2D(f, 3, activation='relu',padding='same')(x)
    outputs = Conv2D(1, 1, activation = 'sigmoid')(x)
    model = Model(inputs=[inputs], outputs = [outputs])

    return model

In [ ]:
model = unet()

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 8)          224       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 256, 256, 8)          584       ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 128, 128, 8)          0         ['conv2d_1[0][0]']            
 D)                                                                                           

# Get Data

In [ ]:
# get data
data_dir = '/content/drive/MyDrive/flood-area-segmentation'
image_paths, mask_paths = create_data(data_dir)

In [ ]:
len(image_paths)

290

In [ ]:
len(mask_paths)

290

# Loading images and masks

In [ ]:
SIZE = 256

# load images and masks
imgs = load_images(image_paths, SIZE)
msks = load_images(mask_paths, SIZE, mask=True)

In [ ]:
imgs

array([[[[0.2165    , 0.2906    , 0.28029999],
         [0.1842    , 0.26769999, 0.2218    ],
         [0.14910001, 0.2141    , 0.2129    ],
         ...,
         [0.0938    , 0.15989999, 0.1446    ],
         [0.1265    , 0.19320001, 0.1882    ],
         [0.0596    , 0.1055    , 0.0977    ]],

        [[0.226     , 0.3193    , 0.32440001],
         [0.19769999, 0.27450001, 0.26989999],
         [0.1531    , 0.2375    , 0.2164    ],
         ...,
         [0.132     , 0.1962    , 0.1521    ],
         [0.1372    , 0.20389999, 0.1996    ],
         [0.0888    , 0.1498    , 0.1427    ]],

        [[0.2097    , 0.28009999, 0.28389999],
         [0.2078    , 0.2784    , 0.28240001],
         [0.1824    , 0.2225    , 0.2516    ],
         ...,
         [0.14399999, 0.2078    , 0.1592    ],
         [0.14129999, 0.2078    , 0.20190001],
         [0.1336    , 0.169     , 0.19599999]],

        ...,

        [[0.1373    , 0.149     , 0.13330001],
         [0.0806    , 0.13519999, 0.0727    ]

In [ ]:
imgs.shape

(290, 256, 256, 3)

In [ ]:
msks

array([[[[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        ...,

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [1.        ],
         [1.        ],
         [1.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [1.        ],
         [1.        ],
         [1.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [1.        ],
         [1.        ],
         [0.73150003]]],


       [[[0.        ],
         [0.        ],
         [0.  

In [ ]:
msks.shape

(290, 256, 256, 1)

# Show sample

In [ ]:
# show sample
show_images(imgs, msks)

# Train the model

In [ ]:
model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
batch_size = 40     # set batch size for training
epochs = 100         # number of all epochs in training
ask_epoch = 101   # number of epochs to run before asking if you want to halt training

In [ ]:
# Config Training
SPE = len(imgs)//batch_size

In [ ]:
# Training
history = model.fit(
    imgs, msks,
    validation_split=0.2,
    epochs=epochs,
    verbose=1,
    steps_per_epoch=SPE,
    batch_size=batch_size
)

Epoch 1/100
7/7 [==============================] - 110s 15s/step - loss: 0.6869 - accuracy: 0.6336 - val_loss: 0.6738 - val_accuracy: 0.7408
Epoch 2/100
7/7 [==============================] - 103s 14s/step - loss: 0.6653 - accuracy: 0.7225 - val_loss: 0.6319 - val_accuracy: 0.7784
Epoch 3/100
7/7 [==============================] - 99s 14s/step - loss: 0.6134 - accuracy: 0.7346 - val_loss: 0.5151 - val_accuracy: 0.7785
Epoch 4/100
7/7 [==============================] - 98s 14s/step - loss: 0.5251 - accuracy: 0.7616 - val_loss: 0.4323 - val_accuracy: 0.8108
Epoch 5/100
7/7 [==============================] - 99s 14s/step - loss: 0.4967 - accuracy: 0.7661 - val_loss: 0.4041 - val_accuracy: 0.8206
Epoch 6/100
7/7 [==============================] - 90s 13s/step - loss: 0.4946 - accuracy: 0.7646 - val_loss: 0.4048 - val_accuracy: 0.8199
Epoch 7/100
7/7 [==============================] - 93s 13s/step - loss: 0.4932 - accuracy: 0.7627 - val_loss: 0.4049 - val_accuracy: 0.8228
Epoch 8/100
7/7 [=

7/7 [==============================] - 71s 10s/step - loss: 0.1495 - accuracy: 0.9241 - val_loss: 0.2960 - val_accuracy: 0.8809


# Plot the history

In [ ]:
plot_training(history)

# Evaluvation

In [ ]:
plt.figure(figsize=(20,25))
n=0
for i in range(1,(5*3)+1):
    plt.subplot(5,3,i)
    if n==0:
        id = np.random.randint(len(imgs))
        image = imgs[id]
        mask = msks[id]
        pred_mask = model.predict(image[np.newaxis,...])

        plt.title("Original Mask")
        show_mask(image, mask)
        n+=1
    elif n==1:
        plt.title("Predicted Mask")
        show_mask(image, pred_mask)
        n+=1
    elif n==2:
        pred_mask = (pred_mask>0.5).astype('float')
        plt.title("Processed Mask")
        show_mask(image, pred_mask)
        n=0
plt.tight_layout()
plt.show()